In [2]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval
!pip install transformers[torch]
!pip install accelerate -U
!pip install accelerate>=0.21.0

In [3]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)
    
    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]
    
    predicted_labels = []
    
    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]
    
    return predicted_labels

In [4]:
# let us try with some example sentences here
sentence = 'उन्होंने कहा, डॉ राजेन्द्र प्रसाद, बाबा साहेब भीमराव अंबेडकर, सरदार पटेल, मौलाना आजाद, सुचेता कृपलानी और अनेक अनगिनत महापुरुषों ने प्रत्यक्ष और अप्रत्यक्ष योगदान देकर ये महान विरासत हमें सौंपी हैं।'

predicted_labels = get_predictions(sentence=sentence, 
                                   tokenizer=tokenizer,
                                   model=model
                                   )

for index in range(len(sentence.split(' '))):
  print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

उन्होंने	O
कहा,	O
डॉ	O
राजेन्द्र	O
प्रसाद,	B-PER
बाबा	I-PER
साहेब	O
भीमराव	O
अंबेडकर,	B-PER
सरदार	I-PER
पटेल,	I-PER
मौलाना	O
आजाद,	B-PER
सुचेता	I-PER
कृपलानी	O
और	B-PER
अनेक	I-PER
अनगिनत	O
महापुरुषों	B-PER
ने	I-PER
प्रत्यक्ष	O
और	O
अप्रत्यक्ष	O
योगदान	O
देकर	O
ये	O
महान	O
विरासत	O
हमें	O
सौंपी	O
हैं।	O


In [5]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam_pr downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam_pr/hi/1.0.0/99b5ec77eabfaa3fbff510d8cf70d7c34519486cb7dbee99ede19474ddff9b20. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
# let's now print how the Dataset looks like
print(raw_datasets)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})


In [7]:
raw_datasets.column_names

{'train': ['tokens', 'ner_tags'],
 'test': ['tokens', 'ner_tags'],
 'validation': ['tokens', 'ner_tags']}

In [8]:
# let's print an instance of dataset
idx=985786
rec=raw_datasets['train'][idx]
for w, t in zip(rec['tokens'],rec['ner_tags']):
  print('{}\t{}'.format(w,t))


उनके	0
27	0
साल	0
के	0
बेटे	0
जीवा	1
को	0
भी	0
दिल	0
की	0
बीमारी	0
थी	0
।	0


In [8]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['tokens', 'ner_tags']
{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)}


In [9]:
text_column_name = "tokens"
label_column_name = "ner_tags"

In [10]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)
print(num_labels)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6}
7


In [11]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indicNER', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indicNER")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indicNER', num_labels=num_labels )

2024-03-13 14:34:34.872706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 14:34:34.872800: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 14:34:35.045854: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/346 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/667M [00:00<?, ?B/s]

In [12]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

In [13]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
ot=len(raw_datasets["train"])
nts=int(ot*0.02)
train_dataset = raw_datasets["train"].select(range(nts))
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/5 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/5 [00:00<?, ?ba/s]

In [15]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/4 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [21]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

In [19]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    learning_rate=4e-5,
    weight_decay=0.01
)

In [22]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
train_result = trainer.train()
metrics = train_result.metrics

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Loc Precision,Loc Recall,Loc F1,Loc Number,Org Precision,Org Recall,Org F1,Org Number,Per Precision,Per Recall,Per F1,Per Number,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.152400,0.170784,0.822301,0.840497,0.831300,10213,0.663565,0.699571,0.681092,9786,0.796245,0.842733,0.818830,10568,0.761905,0.796153,0.778652,0.946342
2,0.109000,0.183289,0.816924,0.849799,0.833037,10213,0.680391,0.682199,0.681294,9786,0.810689,0.831094,0.820764,10568,0.771922,0.789675,0.780698,0.946776
3,0.076200,0.204132,0.813499,0.847351,0.830080,10213,0.672570,0.690783,0.681555,9786,0.804630,0.832040,0.818106,10568,0.765632,0.791932,0.778560,0.946392


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

In [24]:
model.save_pretrained('my_indicNER')

In [25]:
tokenizer.save_pretrained("tokenizer_indicNER")

('tokenizer_indicNER/tokenizer_config.json',
 'tokenizer_indicNER/special_tokens_map.json',
 'tokenizer_indicNER/vocab.txt',
 'tokenizer_indicNER/added_tokens.json',
 'tokenizer_indicNER/tokenizer.json')

In [26]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [27]:
import json

In [28]:
config = json.load(open("/kaggle/working/my_indicNER/config.json"))

In [29]:
config["id2label"]=id2label
config["label2id"]=label2id

In [30]:
json.dump(config, open("/kaggle/working/my_indicNER/config.json","w"))

In [31]:
model = AutoModelForTokenClassification.from_pretrained('/kaggle/working/my_indicNER')

In [32]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [34]:
import torch

def get_ner_from_file(file_path):
    with open(file_path, 'r') as file:
        sentences = file.readlines()
    return get_ner(sentences)

def get_ner(sentences):
    ner_outputs = []
    
    for sentence in sentences:
        tok_sentence = tokenizer(sentence, return_tensors='pt')

        with torch.no_grad():
            logits = model(**tok_sentence).logits.argmax(-1)
            predicted_tokens_classes = [
                model.config.id2label[t.item()] for t in logits[0]]

            predicted_labels = []

            previous_token_id = 0
            word_ids = tok_sentence.word_ids()
            for word_index in range(len(word_ids)):
                if word_ids[word_index] == None:
                    previous_token_id = word_ids[word_index]
                elif word_ids[word_index] == previous_token_id:
                    previous_token_id = word_ids[word_index]
                else:
                    predicted_labels.append(predicted_tokens_classes[word_index])
                    previous_token_id = word_ids[word_index]

            ner_output = []
            for index in range(len(sentence.split(' '))):
                if(index < len(predicted_labels)):
                    ner_output.append((sentence.split(' ')[index], predicted_labels[index]))
                else:
                    ner_output.append((sentence.split(' ')[index], 'O'))
            ner_outputs.append(ner_output)
    
    return ner_outputs

# Example usage
file_path = "/kaggle/input/datakjdk/katare.txt"
ner_results = get_ner_from_file(file_path)
print(ner_results)


[[('\ufeffलगातार', 'O'), ('किसान', 'O'), ('इस', 'O'), ('बाबत', 'O'), ('जेडीए', 'B-ORG'), ('में', 'O'), ('आकर', 'O'), ('योजना', 'O'), ('को', 'O'), ('पूर्ण', 'O'), ('करने', 'O'), ('के', 'O'), ('लिये', 'O'), ('मांग', 'O'), ('कर', 'O'), ('रहे', 'O'), ('है।\n', 'O')], [('राजे', 'B-PER'), ('छह', 'O'), ('राज्यों', 'O'), ('के', 'O'), ('मुख्यमंत्रियों', 'O'), ('के', 'O'), ('साथ', 'O'), ('नीति', 'O'), ('आयोग', 'O'), ('की', 'O'), ('बैठक', 'O'), ('में', 'O'), ('भाग', 'O'), ('लेने', 'O'), ('दिल्ली', 'B-LOC'), ('आईं', 'O'), ('थी।\n', 'O')], [('राजीव', 'B-PER'), ('गाँधी', 'I-PER'), ('ने', 'O'), ('प्रधान', 'O'), ('मंत्री', 'O'), ('कार्यकाल', 'O'), ('के', 'O'), ('दौरान', 'O'), ('उन्हें', 'O'), ('उत्कृष्ट', 'O'), ('कार्यों', 'O'), ('के', 'O'), ('लिए', 'O'), ('पुरस्कृत', 'O'), ('भी', 'O'), ('किया', 'O'), ('था.\n', 'O')], [('कई', 'O'), ('अप्रिय', 'O'), ('घटनाओं', 'O'), ('से', 'O'), ('आपका', 'O'), ('मन', 'O'), ('खिन्न', 'O'), ('रह', 'O'), ('सकता', 'O'), ('है।', 'O'), ('\n', 'O')], [('यही', 'O'), ('पानी', '

In [41]:
!ls

'=0.21.0'       NERTuned_tokenizer.zip	 output_dir	      wandb
 NERTuned.zip   my_indicNER		 tokenizer_indicNER


In [33]:
!zip -r NERTuned.zip /kaggle/working/my_indicNER

  adding: kaggle/working/my_indicNER/ (stored 0%)
  adding: kaggle/working/my_indicNER/config.json (deflated 50%)
  adding: kaggle/working/my_indicNER/model.safetensors (deflated 7%)


In [34]:
!zip -r NERTuned_tokenizer.zip /kaggle/working/tokenizer_indicNER

  adding: kaggle/working/tokenizer_indicNER/ (stored 0%)
  adding: kaggle/working/tokenizer_indicNER/special_tokens_map.json (deflated 80%)
  adding: kaggle/working/tokenizer_indicNER/vocab.txt (deflated 48%)
  adding: kaggle/working/tokenizer_indicNER/tokenizer_config.json (deflated 75%)
  adding: kaggle/working/tokenizer_indicNER/tokenizer.json (deflated 69%)


In [35]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='hi'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

  0%|          | 0/3 [00:00<?, ?it/s]

In [35]:
test_dataset = raw_datasets["test"]
tokenized_test_dataset = test_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on test dataset",
)

Running tokenizer on test dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on test dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [36]:
predictions, labels, metrics = trainer.predict(tokenized_test_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall']=metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1']=metrics[key]

In [37]:
print(metrics)

{'test_loss': 0.17227746546268463, 'test_LOC_precision': 0.802907915993538, 'test_LOC_recall': 0.8094462540716613, 'test_LOC_f1': 0.8061638280616383, 'test_LOC_number': 614, 'test_ORG_precision': 0.644122383252818, 'test_ORG_recall': 0.7619047619047619, 'test_ORG_f1': 0.6980802792321117, 'test_ORG_number': 525, 'test_PER_precision': 0.8307508939213349, 'test_PER_recall': 0.8822784810126583, 'test_PER_f1': 0.8557397176181707, 'test_PER_number': 790, 'test_overall_precision': 0.7667147667147667, 'test_overall_recall': 0.8263348885432866, 'test_overall_f1': 0.7954091816367265, 'test_overall_accuracy': 0.9528980043231288, 'test_runtime': 20.2332, 'test_samples_per_second': 42.85, 'test_steps_per_second': 2.718}


In [38]:
predictions, labels, metrics = trainer.predict(train_dataset)

lang_specific_results = {}
for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall']=metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1']=metrics[key]

In [39]:
print(metrics)

{'test_loss': 0.0533149391412735, 'test_LOC_precision': 0.9109492952101009, 'test_LOC_recall': 0.9348049633235073, 'test_LOC_f1': 0.9227229665719312, 'test_LOC_number': 14587, 'test_ORG_precision': 0.8627958845960141, 'test_ORG_recall': 0.8619896492236918, 'test_ORG_f1': 0.8623925784761425, 'test_ORG_number': 13912, 'test_PER_precision': 0.9095618803965462, 'test_PER_recall': 0.9233216465394105, 'test_PER_f1': 0.9163901150240036, 'test_PER_number': 15402, 'test_overall_precision': 0.8954227804867088, 'test_overall_recall': 0.9077014191020706, 'test_overall_f1': 0.9015202931994029, 'test_overall_accuracy': 0.982339641089361, 'test_runtime': 443.7192, 'test_samples_per_second': 44.431, 'test_steps_per_second': 2.779}
